In [44]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [45]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [46]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [47]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [48]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [83]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
#                     print('wrong')
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [50]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [51]:
file_train = '/home/jindal/notebooks/jindal/NER/language_model/twitter_17-13_train.txt'
file_dev = '/home/jindal/notebooks/jindal/NER/language_model/twitter_17-13_dev.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [52]:
print(len(dev_sentences))

40000


In [53]:
dev_labels[:10]

[2, 10, 10, 130, 55, 49, 4, 7, 32, 2]

In [54]:
print(len(dev_labels))

40000


In [55]:
n_labels = len(labels2Id)
print(n_labels)

1297


In [56]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [57]:
print(dev_sentences[1])

USERMENTION Oh wie lieb ... ich wünsche dir einen guten Rutsch und ein wunderschönes neues Jahr


In [58]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model',
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/jindal/NER/language_model/twitter_emoji_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

            
#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




In [59]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
twitteremojis_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_twitteremojis_1.9m.txt' 
twitteremojis_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitteremojis_1.9m.h5'
batch_size=32*4
twitteremojis_train_file='/home/jindal/notebooks/jindal/NER/language_model/twitter_17-13_train.txt'
tokenize = False
labels2Idx = labels2Id
f1_measure='macro'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [60]:
len(labels2Idx)

1297

In [ ]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels,
           number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs,
           filename_to_log=twitteremojis_pretraining_log, 
           labels_earlier = len(labels2Idx),
           filename_to_save_weigths=twitteremojis_pretraining_save_weigths,
           tokenize= tokenize, 
           labels2Idx= labels2Idx,
          batch_size=batch_size,
           train_file=twitteremojis_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/1
Epoch: 1
Epoch 1/1
14566/14566 [==============================] - 3589s 246ms/step - loss: 4.0196 - acc: 0.2135 - val_loss: 10.9576 - val_acc: 4.7424e-04
Epoch: 2
Epoch 1/1
14566/14566 [==============================] - 3634s 249ms/step - loss: 3.9436 - acc: 0.2193 - val_loss: 11.0780 - val_acc: 3.9936e-04
Epoch: 3
Epoch 1/1
14566/14566 [==============================] - 3580s 246ms/step - loss: 3.9257 - acc: 0.2206 - val_loss: 11.0834 - val_acc: 2.9952e-04
Epoch: 4
Epoch 1/1
 6031/14566 [===========>..................] - ETA: 35:02 - loss: 3.9178 - acc: 0.2214

# TRANSFER LEARNING HERE


In [20]:
def test_model_tl(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log,"w")
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1297)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])
                final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#                 for layer in final_model.layers:
#                     print(layer.name)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
#                 validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
#                                             batch_size = batch_size, check = check_for_generator, 
#                                            labels2Idx = labels2Idx, tokenize = tokenize),
#                 validation_steps = math.ceil(len(dev_labels) / batch_size),
#                 callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [21]:
n_labels =2

In [22]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [23]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [24]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
twitteremojis_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_emojis_1.9m.txt' 
twitteremojis_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_emojis_1.9m.h5'
batch_size=32
twitteremojis_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitteremojis_1.9m.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
labels2Idx = labels2Idx
check_for_generator=3

In [25]:
test_model_tl(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitteremojis_tl_log,
           labels2Idx = labels2Idx,
           filename_to_save_weigths=twitteremojis_tl_save_weigths,
           batch_size=batch_size,
           train_file=twitteremojis_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 37s 281ms/step - loss: 0.5741 - acc: 0.7034
model saved. F1 is 0.560870
Test-Data: Prec: 0.668, Rec: 0.483, F1: 0.561, Acc: 0.750
Epoch: 2
Epoch 1/1
132/132 [==============================] - 35s 265ms/step - loss: 0.5242 - acc: 0.7367
model saved. F1 is 0.588235
Test-Data: Prec: 0.703, Rec: 0.506, F1: 0.588, Acc: 0.766
Epoch: 3
Epoch 1/1
132/132 [==============================] - 35s 261ms/step - loss: 0.5054 - acc: 0.7429
Test-Data: Prec: 0.667, Rec: 0.524, F1: 0.587, Acc: 0.756
Epoch: 4
Epoch 1/1
132/132 [==============================] - 36s 269ms/step - loss: 0.4803 - acc: 0.7654
model saved. F1 is 0.592751
Test-Data: Prec: 0.688, Rec: 0.521, F1: 0.593, Acc: 0.764
Epoch: 5
Epoch 1/1
132/132 [==============================] - 34s 261ms/step - loss: 0.4752 - acc: 0.7680
model saved. F1 is 0.599156
Test-Data: Prec: 0.686, Rec: 0.532, F1: 0.599, Acc: 0.765
Epoch: 6
Epoch 1/1
132/132 [============

132/132 [==============================] - 35s 262ms/step - loss: 0.1177 - acc: 0.9553
Test-Data: Prec: 0.669, Rec: 0.704, F1: 0.686, Acc: 0.787
Epoch: 50
Epoch 1/1
132/132 [==============================] - 35s 263ms/step - loss: 0.0986 - acc: 0.9621
Test-Data: Prec: 0.698, Rec: 0.685, F1: 0.692, Acc: 0.798
prec: 0.7201492537313433 rec: 0.7228464419475655 f1: 0.7214953271028037 acc: 0.8155940594059405 

*****************************************************************************
Test 2/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 37s 282ms/step - loss: 0.5721 - acc: 0.7055
model saved. F1 is 0.506024
Test-Data: Prec: 0.709, Rec: 0.393, F1: 0.506, Acc: 0.746
Epoch: 2
Epoch 1/1
132/132 [==============================] - 35s 269ms/step - loss: 0.5288 - acc: 0.7325
model saved. F1 is 0.533643
Test-Data: Prec: 0.701, Rec: 0.431, F1: 0.534, Acc: 0.751
Epoch: 3
Epoch 1/1
132/132 [==============================] - 35s 263ms/step - loss: 0.5009 - acc: 0.7

132/132 [==============================] - 34s 260ms/step - loss: 0.1154 - acc: 0.9590
Test-Data: Prec: 0.630, Rec: 0.760, F1: 0.689, Acc: 0.774
Epoch: 47
Epoch 1/1
132/132 [==============================] - 34s 260ms/step - loss: 0.1174 - acc: 0.9512
Test-Data: Prec: 0.641, Rec: 0.730, F1: 0.683, Acc: 0.776
Epoch: 48
Epoch 1/1
132/132 [==============================] - 34s 261ms/step - loss: 0.1010 - acc: 0.9612
Test-Data: Prec: 0.640, Rec: 0.727, F1: 0.681, Acc: 0.775
Epoch: 49
Epoch 1/1
132/132 [==============================] - 35s 262ms/step - loss: 0.1064 - acc: 0.9602
Test-Data: Prec: 0.646, Rec: 0.738, F1: 0.689, Acc: 0.780
Epoch: 50
Epoch 1/1
132/132 [==============================] - 35s 263ms/step - loss: 0.1031 - acc: 0.9621
Test-Data: Prec: 0.668, Rec: 0.723, F1: 0.694, Acc: 0.790
prec: 0.6622516556291391 rec: 0.7490636704119851 f1: 0.70298769771529 acc: 0.7908415841584159 

*****************************************************************************
Test 3/5
removing top

132/132 [==============================] - 31s 234ms/step - loss: 0.1452 - acc: 0.9344
Test-Data: Prec: 0.690, Rec: 0.674, F1: 0.682, Acc: 0.792
Epoch: 39
Epoch 1/1
132/132 [==============================] - 31s 232ms/step - loss: 0.1391 - acc: 0.9439
Test-Data: Prec: 0.686, Rec: 0.712, F1: 0.699, Acc: 0.797
Epoch: 40
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.1288 - acc: 0.9496
Test-Data: Prec: 0.631, Rec: 0.783, F1: 0.699, Acc: 0.777
Epoch: 41
Epoch 1/1
132/132 [==============================] - 30s 230ms/step - loss: 0.1406 - acc: 0.9437
Test-Data: Prec: 0.664, Rec: 0.712, F1: 0.687, Acc: 0.786
Epoch: 42
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.1324 - acc: 0.9486
Test-Data: Prec: 0.678, Rec: 0.700, F1: 0.689, Acc: 0.791
Epoch: 43
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.1210 - acc: 0.9553
Test-Data: Prec: 0.677, Rec: 0.708, F1: 0.692, Acc: 0.792
Epoch: 44
Epoch 1/1
132/132 [=

# gradual unfreezing

In [62]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [77]:
n_labels=2

In [78]:
print(dev_sentences[0])
print(len(dev_sentences))
print(len(dev_labels))

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
808
808


In [85]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1297)

        # transfer learning
        if load_model_weights and model_weights_file:
                print("loading weights")
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx,tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
#         f.close()
#         f = open(filename_to_log, 'a', encoding='utf-8')
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)
#     f.close()

In [86]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (16,18), (3,18), (1,18)]

strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]

In [ ]:
for i in range(len(strings)):

    string = strings[i]
    print("approach: %s" %(string))
    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 5
    number_of_epochs = 50
    labels2Id = labels2Idx
    twitteremojis_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitteremojis1.9m_1297_'+string+'.txt'
    print("log file: %s" %(twitteremojis_tl_log))
    twitteremojis_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitteremojis1.9m_1297_' +string+'.h5'
    print("save h5 file: %s\n\n" %(twitteremojis_tl_save_weigths))
    batch_size=32
    twitteremojis_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
    f1_measure='binary'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitteremojis_1.9m.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3

    test_model_tl_unfreezing(generator=generator, 
               train_sentences=train_sentences, 
               devLabels=devLabels, 
               number_of_tests= number_of_tests,
               number_of_epochs=number_of_epochs, 
               filename_to_log=twitteremojis_tl_log, 
               labels2Idx = labels2Id,
               filename_to_save_weigths=twitteremojis_tl_save_weigths,
               batch_size=batch_size,
               unfreezing_strategy = strategy,       
               train_file=twitteremojis_tl_train_file, 
               f1_measure=f1_measure, 
               pos_label=pos_label, 
               load_model_weights=load_model_weights,
               model_weights_file = model_weights_file, 
               nb_sequence_length=nb_sequence_length, 
               nb_embedding_dims=nb_embedding_dims, 
               check_for_generator= 3)

approach: suf_bu
log file: /home/jindal/notebooks/jindal/NER/language_model/results_tl_twitteremojis1.9m_1297_suf_bu.txt
save h5 file: /home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitteremojis1.9m_1297_suf_bu.h5


[(18, 18), (17, 16), (15, 3), (2, 1), (18, 1)]
Test 1/5
loading weights
removing top layer
unfreezing dense_53
---------------------------------------
0 input_19 False
1 bidirectional_19 False
2 leaky_re_lu_91 False
3 conv1d_55 False
4 conv1d_56 False
5 conv1d_57 False
6 leaky_re_lu_92 False
7 leaky_re_lu_93 False
8 leaky_re_lu_94 False
9 global_max_pooling1d_55 False
10 global_max_pooling1d_56 False
11 global_max_pooling1d_57 False
12 dropout_55 False
13 dropout_56 False
14 dropout_57 False
15 concatenate_19 False
16 dense_51 False
17 leaky_re_lu_95 False
18 dense_53 True
Epoch: 1/50
Epoch 1/1
132/132 [==============================] - 13s 99ms/step - loss: 0.6295 - acc: 0.6598
model saved. F1 is 0.324022
Test-Data: Prec: 0.637, Rec: 0.217, F1: 0

132/132 [==============================] - 10s 77ms/step - loss: 0.5584 - acc: 0.7116
Test-Data: Prec: 0.673, Rec: 0.386, F1: 0.490, Acc: 0.735
Epoch: 45/50
Epoch 1/1
132/132 [==============================] - 11s 83ms/step - loss: 0.5592 - acc: 0.7095
model saved. F1 is 0.508009
Test-Data: Prec: 0.653, Rec: 0.416, F1: 0.508, Acc: 0.734
Epoch: 46/50
Epoch 1/1
132/132 [==============================] - 10s 79ms/step - loss: 0.5650 - acc: 0.7057
Test-Data: Prec: 0.656, Rec: 0.386, F1: 0.486, Acc: 0.730
Epoch: 47/50
Epoch 1/1
132/132 [==============================] - 11s 81ms/step - loss: 0.5627 - acc: 0.7057
Test-Data: Prec: 0.667, Rec: 0.352, F1: 0.461, Acc: 0.728
Epoch: 48/50
Epoch 1/1
132/132 [==============================] - 11s 83ms/step - loss: 0.5606 - acc: 0.7081
Test-Data: Prec: 0.705, Rec: 0.367, F1: 0.483, Acc: 0.740
Epoch: 49/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.5649 - acc: 0.7083
Test-Data: Prec: 0.660, Rec: 0.371, F1: 0.475, Acc:

132/132 [==============================] - 10s 78ms/step - loss: 0.5452 - acc: 0.7218
Test-Data: Prec: 0.703, Rec: 0.453, F1: 0.551, Acc: 0.756
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.5426 - acc: 0.7147
Test-Data: Prec: 0.694, Rec: 0.449, F1: 0.545, Acc: 0.752
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.5373 - acc: 0.7237
Test-Data: Prec: 0.711, Rec: 0.442, F1: 0.545, Acc: 0.756
Epoch: 42/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5370 - acc: 0.7251
Test-Data: Prec: 0.703, Rec: 0.453, F1: 0.551, Acc: 0.756
Epoch: 43/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.5298 - acc: 0.7256
Test-Data: Prec: 0.684, Rec: 0.479, F1: 0.564, Acc: 0.755
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 10s 79ms/step - loss: 0.5377 - acc: 0.7273
Test-Data: Prec: 0.684, Rec: 0.494, F1: 0.574, Acc: 0.757
Epoch: 45/50
Epoch 1/

132/132 [==============================] - 11s 83ms/step - loss: 0.3577 - acc: 0.8411
Test-Data: Prec: 0.679, Rec: 0.569, F1: 0.619, Acc: 0.769
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 11s 82ms/step - loss: 0.3655 - acc: 0.8383
Test-Data: Prec: 0.703, Rec: 0.551, F1: 0.618, Acc: 0.775
Epoch: 36/50
Epoch 1/1
132/132 [==============================] - 11s 80ms/step - loss: 0.3495 - acc: 0.8461
Test-Data: Prec: 0.708, Rec: 0.562, F1: 0.626, Acc: 0.778
Epoch: 37/50
Epoch 1/1
132/132 [==============================] - 11s 82ms/step - loss: 0.3519 - acc: 0.8464
Test-Data: Prec: 0.699, Rec: 0.566, F1: 0.625, Acc: 0.776
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.3345 - acc: 0.8525
Test-Data: Prec: 0.685, Rec: 0.596, F1: 0.637, Acc: 0.776
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 11s 83ms/step - loss: 0.3302 - acc: 0.8537
Test-Data: Prec: 0.680, Rec: 0.614, F1: 0.646, Acc: 0.777
Epoch: 42/50
Epoch 1/

132/132 [==============================] - 31s 236ms/step - loss: 0.2393 - acc: 0.8984
Test-Data: Prec: 0.704, Rec: 0.685, F1: 0.694, Acc: 0.801
Epoch: 31/50
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.2415 - acc: 0.8975
Test-Data: Prec: 0.713, Rec: 0.652, F1: 0.681, Acc: 0.798
Epoch: 32/50
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.2420 - acc: 0.8963
Test-Data: Prec: 0.700, Rec: 0.682, F1: 0.691, Acc: 0.798
Epoch: 33/50
Epoch 1/1
132/132 [==============================] - 31s 235ms/step - loss: 0.2353 - acc: 0.9020
Test-Data: Prec: 0.719, Rec: 0.670, F1: 0.694, Acc: 0.804
Epoch: 34/50
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.2235 - acc: 0.9077
Test-Data: Prec: 0.707, Rec: 0.685, F1: 0.696, Acc: 0.802
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 31s 232ms/step - loss: 0.2324 - acc: 0.8982
Test-Data: Prec: 0.708, Rec: 0.670, F1: 0.688, Acc: 0.800
Epoch: 36/50
Ep

...

132/132 [==============================] - 10s 77ms/step - loss: 0.5565 - acc: 0.7135
Test-Data: Prec: 0.683, Rec: 0.363, F1: 0.474, Acc: 0.734
Epoch: 17/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.5622 - acc: 0.7088
model saved. F1 is 0.481752
Test-Data: Prec: 0.688, Rec: 0.371, F1: 0.482, Acc: 0.736
Epoch: 18/50
Epoch 1/1
132/132 [==============================] - 10s 79ms/step - loss: 0.5531 - acc: 0.7150
Test-Data: Prec: 0.669, Rec: 0.371, F1: 0.477, Acc: 0.731
Epoch: 19/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.5588 - acc: 0.7119
model saved. F1 is 0.485577
Test-Data: Prec: 0.678, Rec: 0.378, F1: 0.486, Acc: 0.735
Epoch: 20/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.5679 - acc: 0.7005
Test-Data: Prec: 0.673, Rec: 0.378, F1: 0.484, Acc: 0.734
Epoch: 21/50
Epoch 1/1
132/132 [==============================] - 11s 81ms/step - loss: 0.5651 - acc: 0.7029
Test-Data: Prec: 0.664,

132/132 [==============================] - 11s 80ms/step - loss: 0.5539 - acc: 0.7152
Test-Data: Prec: 0.691, Rec: 0.393, F1: 0.501, Acc: 0.741
Epoch: 12/50
Epoch 1/1
132/132 [==============================] - 11s 80ms/step - loss: 0.5450 - acc: 0.7188
Test-Data: Prec: 0.677, Rec: 0.423, F1: 0.521, Acc: 0.743
Epoch: 13/50
Epoch 1/1
132/132 [==============================] - 10s 79ms/step - loss: 0.5600 - acc: 0.7090
Test-Data: Prec: 0.714, Rec: 0.431, F1: 0.537, Acc: 0.755
Epoch: 14/50
Epoch 1/1
132/132 [==============================] - 10s 79ms/step - loss: 0.5450 - acc: 0.7235
Test-Data: Prec: 0.738, Rec: 0.412, F1: 0.529, Acc: 0.757
Epoch: 15/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.5466 - acc: 0.7171
Test-Data: Prec: 0.723, Rec: 0.401, F1: 0.516, Acc: 0.751
Epoch: 16/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.5518 - acc: 0.7157
Test-Data: Prec: 0.678, Rec: 0.449, F1: 0.541, Acc: 0.748
Epoch: 17/50
Epoch 1/

132/132 [==============================] - 10s 79ms/step - loss: 0.4905 - acc: 0.7540
Test-Data: Prec: 0.685, Rec: 0.513, F1: 0.587, Acc: 0.761
Epoch: 7/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.4817 - acc: 0.7616
model saved. F1 is 0.612420
Test-Data: Prec: 0.715, Rec: 0.536, F1: 0.612, Acc: 0.776
Epoch: 8/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.4675 - acc: 0.7718
model saved. F1 is 0.620833
Test-Data: Prec: 0.700, Rec: 0.558, F1: 0.621, Acc: 0.775
Epoch: 9/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.4766 - acc: 0.7637
Test-Data: Prec: 0.716, Rec: 0.509, F1: 0.595, Acc: 0.771
Epoch: 10/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.4611 - acc: 0.7756
model saved. F1 is 0.625259
Test-Data: Prec: 0.699, Rec: 0.566, F1: 0.625, Acc: 0.776
Epoch: 11/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.4566 - acc: 0.782

132/132 [==============================] - 35s 264ms/step - loss: 0.4120 - acc: 0.8045
Test-Data: Prec: 0.703, Rec: 0.603, F1: 0.649, Acc: 0.785
Epoch: 2/50
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.3942 - acc: 0.8182
Test-Data: Prec: 0.697, Rec: 0.622, F1: 0.657, Acc: 0.786
Epoch: 3/50
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.3863 - acc: 0.8134
model saved. F1 is 0.675781
Test-Data: Prec: 0.706, Rec: 0.648, F1: 0.676, Acc: 0.795
Epoch: 4/50
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.3859 - acc: 0.8168
Test-Data: Prec: 0.712, Rec: 0.622, F1: 0.664, Acc: 0.792
Epoch: 5/50
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.3752 - acc: 0.8243
model saved. F1 is 0.679463
Test-Data: Prec: 0.697, Rec: 0.663, F1: 0.679, Acc: 0.793
Epoch: 6/50
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.3725 - acc: 0.8295
model saved. F1 is 0.6

132/132 [==============================] - 30s 229ms/step - loss: 0.2181 - acc: 0.9110
Test-Data: Prec: 0.752, Rec: 0.693, F1: 0.721, Acc: 0.823
Epoch: 49/50
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.2058 - acc: 0.9136
Test-Data: Prec: 0.707, Rec: 0.704, F1: 0.705, Acc: 0.806
Epoch: 50/50
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.2011 - acc: 0.9195
Test-Data: Prec: 0.724, Rec: 0.708, F1: 0.716, Acc: 0.814
unfreezing dense_62
---------------------------------------
0 input_22 False
1 bidirectional_22 True
2 leaky_re_lu_106 True
3 conv1d_64 True
4 conv1d_65 True
5 conv1d_66 True
6 leaky_re_lu_107 True
7 leaky_re_lu_108 True
8 leaky_re_lu_109 True
9 global_max_pooling1d_64 True
10 global_max_pooling1d_65 True
11 global_max_pooling1d_66 True
12 dropout_64 True
13 dropout_65 True
14 dropout_66 True
15 concatenate_22 True
16 dense_60 True
17 leaky_re_lu_110 True
18 dense_62 True
Epoch: 1/50
Epoch 1/1
132/132 [===========